In [4]:
import pandas as pd
import mysql.connector

In [5]:

restaurants_df = pd.read_csv("Bangalore_Restaurants.csv")
print(restaurants_df)

           Restaurant_Name                                           Category  \
0              Burma Burma  Asian, Burmese, Bubble Tea, Salad, Tea, Desser...   
1     Windmills Craftworks  Continental, Fast Food, Kebab, Beverages, Ital...   
2           CTR Shri Sagar                                       South Indian   
3     Brahmin's Coffee Bar                                       South Indian   
4         Milano Ice Cream                     Desserts, Ice Cream, Beverages   
...                    ...                                                ...   
5104     Chopstick Kitchen                            Chinese, Momos, Seafood   
5105         Punjabi Dhaba                                       North Indian   
5106  Abhiruchi Restaurant  Biryani, Andhra, North Indian, Chinese, Hydera...   
5107      The Q Restaurant                      North Indian, Chinese, Kerala   
5108      Flavors Of India           North Indian, Chinese, Andhra, Fast Food   

      Pricing_for_2        

In [6]:
print(restaurants_df.head())


        Restaurant_Name                                           Category  \
0           Burma Burma  Asian, Burmese, Bubble Tea, Salad, Tea, Desser...   
1  Windmills Craftworks  Continental, Fast Food, Kebab, Beverages, Ital...   
2        CTR Shri Sagar                                       South Indian   
3  Brahmin's Coffee Bar                                       South Indian   
4      Milano Ice Cream                     Desserts, Ice Cream, Beverages   

   Pricing_for_2                          Locality  Dining_Rating  \
0           1500            Indiranagar, Bangalore            4.9   
1           2500   Windmills Craftworks, Bangalore            4.9   
2            150           Malleshwaram, Bangalore            4.9   
3            100           Basavanagudi, Bangalore            4.9   
4            400            Indiranagar, Bangalore            4.9   

   Dining_Review_Count  Delivery_Rating  Delivery_Rating_Count  \
0               2790.0              4.5           

In [7]:
import mysql.connector

In [19]:
db = mysql.connector.connect(host="localhost",user="root",password="Logesh",database="zomoto")
cursor = db.cursor()
print(db)

In [20]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS Restaurants (
        id INT AUTO_INCREMENT PRIMARY KEY,
        restaurant_name VARCHAR(255),
        latitude FLOAT,
        longitude FLOAT
    )
""")
#cursor.execute(create_table_query)
db.commit()

print("✅ Table 'restaurants' is ready!")


✅ Table 'restaurants' is ready!


In [23]:
print(restaurants_df.columns)


Index(['Restaurant_Name', 'Category', 'Pricing_for_2', 'Locality',
       'Dining_Rating', 'Dining_Review_Count', 'Delivery_Rating',
       'Delivery_Rating_Count', 'Website', 'Address', 'Phone_No', 'Latitude',
       'Longitude'],
      dtype='object')


In [29]:

for _, row in restaurants_df.iterrows():
    cursor.execute(
        "INSERT INTO Restaurants (restaurant_name, latitude, longitude) VALUES (%s, %s, %s)",
        (row['restaurant_name'], row['latitude'], row['longitude'])
    )



In [30]:
db.commit()
print("Data inserted successfully!")


Data inserted successfully!


In [34]:
home_lat = 12.9716  
home_lon = 77.5946 

In [35]:
query = """
    SELECT 
        restaurant_name,
        latitude,
        longitude,
        ROUND(111.1 * SQRT(POW(latitude - %s, 2) + POW(longitude - %s, 2)), 0) AS distance_km,
        RANK() OVER (ORDER BY distance_km ASC) AS rank_distance,
        DENSE_RANK() OVER (ORDER BY distance_km ASC) AS dense_rank_distance
    FROM Restaurants
    ORDER BY distance_km;
"""


In [39]:
query = """
    SELECT 
        restaurant_name,
        latitude,
        longitude,
        ROUND(111.1 * SQRT(POW(latitude - %s, 2) + POW(longitude - %s, 2)), 0) AS distance_km,
        RANK() OVER (ORDER BY ROUND(111.1 * SQRT(POW(latitude - %s, 2) + POW(longitude - %s, 2)), 0) ASC) AS rank_distance,
        DENSE_RANK() OVER (ORDER BY ROUND(111.1 * SQRT(POW(latitude - %s, 2) + POW(longitude - %s, 2)), 0) ASC) AS dense_rank_distance
    FROM Restaurants
    ORDER BY distance_km;
"""

cursor.execute(query, (home_lat, home_lon, home_lat, home_lon, home_lat, home_lon))
results = cursor.fetchall()

In [40]:
columns = ["restaurant_name", "latitude", "longitude", "distance_km", "rank_distance", "dense_rank_distance"]
df_results = pd.DataFrame(results, columns=columns)
print(df_results.head(10))

                restaurant_name  latitude  longitude  distance_km  \
0                         Shiro   12.9718    77.5965          0.0   
1          SodaBottleOpenerWala   12.9706    77.5976          0.0   
2                       Toscano   12.9718    77.5959          0.0   
3  Alba - JW Marriott Bengaluru   12.9719    77.5951          0.0   
4                         Skyye   12.9716    77.5962          0.0   
5                    Farzi Cafe   12.9723    77.5962          0.0   
6                The Biere Club   12.9710    77.5975          0.0   
7                      Rasovara   12.9720    77.5964          0.0   
8              Glen's Bakehouse   12.9698    77.5975          0.0   
9                       Sanchez   12.9719    77.5962          0.0   

   rank_distance  dense_rank_distance  
0              1                    1  
1              1                    1  
2              1                    1  
3              1                    1  
4              1                    1  


In [45]:
df_results.to_csv(r"C:\Users\loges\Downloads\restraunt\nearest restaurant.csv", index=False)
print("Results saved successfully! 🎉")


Results saved successfully! 🎉
